In [11]:
print(
"""
************************************************************
Author: Shota DEGUCHI
        Strucural Analysis Lab. Kyushu Uni. (July 19, 2021)
************************************************************
""")


import sys
import os
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import scipy.io
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
# import seaborn as sns
import time
import datetime

print("current python version:", sys.version)
print("        tf     version:", tf.__version__)


************************************************************
Author: Shota DEGUCHI
        Strucural Analysis Lab. Kyushu Uni. (July 19, 2021)
************************************************************

current python version: 3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]
        tf     version: 2.5.0


In [12]:
############################################################
################      HYPER PARAMETERS      ################
############################################################

lr      = 1e-3
c_tol   = 1e-3
n_btch  = 2 ** 7   # 2 ** x: 5 => 32, 6 => 64, 7 => 128, 8 => 256
n_epch  = int(5e2)
f_mntr  = int(1e1)
layers  = [3] + 7 * [20] + [3]
gpu_flg = 0   # 0, 1, 2, or 4 (see: https://www.tensorflow.org/guide/gpu)
              # 0: Restrict TensorFlow to only use the first GPU
              # 1: Currently, memory growth needs to be the same across GPUs
              # 2: Create 2 virtual GPUs with 1GB memory each
              # 4: Create 4 virtual GPUs with 1GB memory each

seed    = 1234
np.random.seed(seed)
tf.random.set_seed(seed)

In [13]:
U_star = pd.read_csv('../../../data/asymmetric_squares/U_arrange_data.csv', header = None, sep = '\s+').values
P_star = pd.read_csv('../../../data/asymmetric_squares/p_arrange_data.csv', header = None, sep = '\s+').values
t0, t1, dt =  0., 20,  .1; t_star = np.arange  (t0, t1, dt); t_star = t_star.reshape(-1, 1)
x0, x1, nx =  1.,  8., 99; x_star = np.linspace(x0, x1, nx)
y0, y1, ny = -2.,  2., 50; y_star = np.linspace(y0, y1, ny)
x_star, y_star = np.meshgrid(x_star, y_star); x_star, y_star = x_star.reshape(-1, 1), y_star.reshape(-1, 1)
X_star = np.c_[x_star, y_star]

N = X_star.shape[0]; T = t_star.shape[0]
XX = np.tile(X_star[:,0:1], (1, T)); YY = np.tile(X_star[:,1:2], (1, T)); TT = np.tile(t_star, (1, N)).T
UU = U_star[:,0].reshape(T, N).T; VV = - U_star[:,1].reshape(T, N).T; PP = P_star.reshape(T, N).T

ns_lvl = .20
UU_ns = UU + ns_lvl * np.std(UU) * np.random.randn(UU.shape[0], UU.shape[1])
VV_ns = VV + ns_lvl * np.std(VV) * np.random.randn(VV.shape[0], VV.shape[1])
PP_ns = PP + ns_lvl * np.std(PP) * np.random.randn(PP.shape[0], PP.shape[1])

x = XX.flatten()[:,None]; y = YY.flatten()[:,None]; t = TT.flatten()[:,None]
u = UU.flatten()[:,None]; v = VV.flatten()[:,None]; p = PP.flatten()[:,None]
u_ns = UU_ns.flatten()[:,None]; v_ns = VV_ns.flatten()[:,None]; p_ns = PP_ns.flatten()[:,None]

In [19]:
N_trn_frac = .007
N_val_frac = .003
N_trn = int(N_trn_frac * N * T)
N_val = int(N_val_frac * N * T)
# N_trn = int(300)
# N_val = int(300)

N_all = N_trn + N_val

idx_all = np.random.choice(N * T, N_all, replace = False)

idx_trn = idx_all[0 : N_trn]
idx_val = idx_all[N_trn : N_all]

tm = 10; tm = np.array([tm])

u0 = -.3; u1 = 1.5;  ut = .3
v0 = -.6; v1 = .6 ;  vt = .3
p0 = -.6; p1 = .2;   pt = .2

idx_trn = np.random.choice(N * T, N_trn, replace = False)
x_trn    = x[idx_trn,:];    y_trn    = y[idx_trn,:];    t_trn    = t[idx_trn,:]
u_trn    = u[idx_trn,:];    v_trn    = v[idx_trn,:];    p_trn    = p[idx_trn,:]
u_trn_ns = u_ns[idx_trn,:]; v_trn_ns = v_ns[idx_trn,:]; p_trn_ns = p_ns[idx_trn,:]

idx_val = np.random.choice(N * T, N_val, replace = False)
x_val    = x[idx_val,:];    y_val    = y[idx_val,:];    t_val    = t[idx_val,:]
u_val    = u[idx_val,:];    v_val    = v[idx_val,:];    p_val    = p[idx_val,:]
u_val_ns = u_ns[idx_val,:]; v_val_ns = v_ns[idx_val,:]; p_val_ns = p_ns[idx_val,:]

print("N_trn:", N_trn)
print("N_val:", N_val)

# plt.scatter(x_trn, y_trn, marker = ".")
# plt.scatter(x_val, y_val, marker = ".", alpha = .3)
# plt.show()

N_trn: 69300
N_val: 29700


In [20]:
from tensorflow.keras.layers import Dense, InputLayer
from datetime import datetime
from tensorflow.summary import SummaryWriter

class Shiba_PINN(tf.keras.Model):
    def __init__(self,
                x_trn, y_trn, t_trn, u_trn, v_trn, 
                x_val, y_val, t_val, u_val, v_val,  
                w_init="Glorot", h_num=40, out_num=2, 
                w_prd=1., w_pde=1., lr=1e-3, activ="tanh", opt="Adam"):
        super(Shiba_PINN, self).__init__()

        init_weight = self.weight_initializer(w_init)

        self._dtype = tf.float32
        self.activ = activ

        # training set
        self.x_trn = x_trn; self.y_trn = y_trn; self.t_trn = t_trn
        self.u_trn = u_trn; self.v_trn = v_trn
        
        # validation set
        self.x_val = x_val; self.y_val = y_val; self.t_val = t_val
        self.u_val = u_val; self.v_val = v_val

        self.lr = lr
        self.optimizer = self.get_optimizer(opt) # 最適化手法

        self.rhoi = tf.constant([1],   dtype = self._dtype)
        self.nu   = tf.constant([.01], dtype = self._dtype)

        # weight for loss terms
        self.w_prd = w_prd
        self.w_pde = w_pde

        # self.inputs  = InputLayer(name='inputs', input_shape=(input_num,))                                        # 入力層は省略可
        # self.layer1  = Dense(name='layer1' , kernel_initializer="glorot_normal", bias_initializer='zeros', units=h_num  )
        self.layer1  = Dense(name='layer1' , kernel_initializer=init_weight, bias_initializer='zeros', units=h_num  )
        self.layer2  = Dense(name='layer2' , kernel_initializer=init_weight, bias_initializer='zeros', units=h_num  )
        self.layer3  = Dense(name='layer3' , kernel_initializer=init_weight, bias_initializer='zeros', units=h_num  )
        self.layer4  = Dense(name='layer4' , kernel_initializer=init_weight, bias_initializer='zeros', units=h_num  )
        self.layer5  = Dense(name='layer5' , kernel_initializer=init_weight, bias_initializer='zeros', units=h_num  )
        self.layer6  = Dense(name='layer6' , kernel_initializer=init_weight, bias_initializer='zeros', units=h_num  )
        self.layer7  = Dense(name='layer7' , kernel_initializer=init_weight, bias_initializer='zeros', units=h_num  )
        self.outputs = Dense(name='outputs', kernel_initializer=init_weight, bias_initializer='zeros', units=out_num)
    
    def weight_initializer(self, w_init):
        if   w_init == "Glorot":
            return tf.keras.initializers.GlorotNormal()
        elif w_init == "He":
            return  tf.keras.initializers.HeNormal()
        elif w_init == "LeCun":
            return tf.keras.initializers.LecunNormal()
        else:
            raise Exception(">>>>> Exception: weight initializer not specified correctly")

    def get_activation(self, X):
        if self.activ == "tanh":
            return tf.keras.activations.tanh(X)
        elif self.activ == "gelu":
            return tf.keras.activations.gelu(X)
        elif self.activ == "silu" or self.activ == "swish-1":
            return tf.keras.activations.swish(X)
        else:
            raise Exception(">>>>> Exception: activation function not specified correctly")
    
    def get_optimizer(self, opt_name):
        if   opt_name == "SGD":
            optimizer = tf.keras.optimizers.SGD(learning_rate = self.lr, momentum = 0.0, nesterov = False)
        elif opt_name == "Adadelta":
            optimizer = tf.keras.optimizers.Adadelta(learning_rate = self.lr, rho = 0.95)
        elif opt_name == "Adagrad":
            optimizer = tf.keras.optimizers.Adagrad(learning_rate = self.lr, initial_accumulator_value = 0.1)
        elif opt_name == "RMSprop":
            optimizer = tf.keras.optimizers.RMSprop(learning_rate = self.lr, rho = 0.9, momentum = 0.0, centered = False)
        elif opt_name == "Adam":
            optimizer = tf.keras.optimizers.Adam(learning_rate = self.lr, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
        elif opt_name == "Adamax":
            optimizer = tf.keras.optimizers.Adamax(learning_rate = self.lr, beta_1 = 0.9, beta_2 = 0.999)
        elif opt_name == "Nadam":
            optimizer = tf.keras.optimizers.Nadam(learning_rate = self.lr, beta_1 = 0.9, beta_2 = 0.999)
        else:
            raise Exception(">>>>> Exception: optimizer not specified correctly")
            
        return optimizer

    # @tf.function
    def call(self, X):
        # X = self.inputs(X)                        #入力層は省略可 
        X = self.get_activation(self.layer1(X))
        X = self.get_activation(self.layer2(X))
        X = self.get_activation(self.layer3(X))
        X = self.get_activation(self.layer4(X))
        X = self.get_activation(self.layer5(X))
        X = self.get_activation(self.layer6(X))
        X = self.get_activation(self.layer7(X))
        out = self.get_activation(self.outputs(X))
        return out

    def PDE(self, x, y, t):
        rhoi = self.rhoi
        nu   = self.nu
        x = tf.convert_to_tensor(x, dtype = self._dtype)
        y = tf.convert_to_tensor(y, dtype = self._dtype)
        t = tf.convert_to_tensor(t, dtype = self._dtype)

        # print(x)
        with tf.GradientTape(persistent = True) as tp1:
            tp1.watch(x)
            tp1.watch(y)
            tp1.watch(t)

            u_v_p = self.call(tf.concat([x, y, t], 1))
            u = u_v_p[:,0:1]
            v = u_v_p[:,1:2]
            p = u_v_p[:,2:3]
            
            u_x = tp1.gradient(u, x); u_y = tp1.gradient(u, y)
            v_x = tp1.gradient(v, x); v_y = tp1.gradient(v, y)
        
        u_t  = tp1.gradient(u, t)  ; v_t = tp1.gradient(v, t)
        p_x  = tp1.gradient(p, x)  ; p_y = tp1.gradient(p, y)
        u_xx = tp1.gradient(u_x, x); u_yy = tp1.gradient(u_y, y)
        v_xx = tp1.gradient(v_x, x); v_yy = tp1.gradient(v_y, y)
        # del tp1
        gv_c = u_x + v_y                                                   # continuity
        gv_x = u_t + u * u_x + v * u_y + rhoi * p_x - nu * (u_xx + u_yy)   # momentum
        gv_y = v_t + u * v_x + v * v_y + rhoi * p_y - nu * (v_xx + v_yy)
        return u, v, p, gv_c, gv_x, gv_y

    def loss_glb(self, x, y, t, u, v):
        u_hat, v_hat, p_hat, gv_c_hat, gv_x_hat, gv_y_hat = self.PDE(x, y, t)
        loss_prd =    tf.reduce_mean(tf.square(u - u_hat)) \
                    + tf.reduce_mean(tf.square(v - v_hat))
        loss_prd = self.w_prd * loss_prd
        loss_pde =    tf.reduce_mean(tf.square(gv_c_hat)) \
                    + tf.reduce_mean(tf.square(gv_x_hat)) \
                    + tf.reduce_mean(tf.square(gv_y_hat))
        loss_pde = self.w_pde * loss_pde
        loss_glb = loss_prd + loss_pde
        return loss_glb
    
    @tf.function
    def loss_grad(self, x, y, t, u, v):
        with tf.GradientTape() as tp:
            loss = self.loss_glb(x, y, t, u, v)
        grad = tp.gradient(loss, self.trainable_variables)
        return loss, grad

    # @tf.function
    def minibatch_train(self, batch):
        n_data  = self.x_trn.shape[0]
        sff_idx = np.random.permutation(n_data)
        for idx in range(0, n_data, batch):
            x_trn_batch = self.x_trn[sff_idx[idx: idx + batch if idx + batch < n_data else n_data]]
            y_trn_batch = self.y_trn[sff_idx[idx: idx + batch if idx + batch < n_data else n_data]]
            t_trn_batch = self.t_trn[sff_idx[idx: idx + batch if idx + batch < n_data else n_data]]
            u_trn_batch = self.u_trn[sff_idx[idx: idx + batch if idx + batch < n_data else n_data]]
            v_trn_batch = self.v_trn[sff_idx[idx: idx + batch if idx + batch < n_data else n_data]]
            x_trn_batch = tf.convert_to_tensor(x_trn_batch, dtype = self._dtype)
            y_trn_batch = tf.convert_to_tensor(y_trn_batch, dtype = self._dtype)
            t_trn_batch = tf.convert_to_tensor(t_trn_batch, dtype = self._dtype)
            u_trn_batch = tf.convert_to_tensor(u_trn_batch, dtype = self._dtype)
            v_trn_batch = tf.convert_to_tensor(v_trn_batch, dtype = self._dtype)
            loss_trn, grad_trn = self.loss_grad(x_trn_batch, y_trn_batch, t_trn_batch, u_trn_batch, v_trn_batch)
            self.optimizer.apply_gradients(zip(grad_trn, self.trainable_variables))

        return loss_trn
    
    def fullbatch_train(self):
        x_trn = tf.convert_to_tensor(self.x_trn, dtype = self._dtype)
        y_trn = tf.convert_to_tensor(self.y_trn, dtype = self._dtype)
        t_trn = tf.convert_to_tensor(self.t_trn, dtype = self._dtype)
        u_trn = tf.convert_to_tensor(self.u_trn, dtype = self._dtype)
        v_trn = tf.convert_to_tensor(self.v_trn, dtype = self._dtype)
        loss_trn, grad_trn = self.loss_grad(x_trn, y_trn, t_trn, u_trn, v_trn)
        self.optimizer.apply_gradients(zip(grad_trn, self.trainable_variables))
        return loss_trn

    def train(self, epoch = 10 ** 5, batch = 693, tol = 1e-5): 
        # logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        t0 = time.time()
        # with tf.profiler.experimental.Profile(logs):
        for ep in range(epoch):

            # minibatch 学習
            ep_loss_trn = self.minibatch_train(batch)
            # fullibatch 学習
            # ep_loss_trn = self.fullbatch_train()

            if ep % f_mntr == 0:
                elps = time.time() - t0
                print("ep: %d, loss_trn: %.6e, elps: %.3f" % (ep, ep_loss_trn, elps))
                t0 = time.time()
                
            if ep_loss_trn < tol:
                print(">>>>> program terminating with the loss converging to its tolerance.")
                break

In [21]:
Shiba_PINN = Shiba_PINN(x_trn, y_trn, t_trn, u_trn, v_trn, 
               x_val, y_val, t_val, u_val, v_val)

In [23]:
print("learning rate  :", lr)
print("# of epoch     :", n_epch)
print("batch size     :", n_btch)
print("convergence tol:", c_tol)
print("N_trn          :", N_trn)
print("N_val          :", N_val)

with tf.device("GPU:0"):
    Shiba_PINN.train(epoch = n_epch, tol = c_tol)

learning rate  : 0.001
# of epoch     : 500
batch size     : 128
convergence tol: 0.001
N_trn          : 69300
N_val          : 29700
ep: 0, loss_trn: 2.523114e-01, elps: 1.042
ep: 10, loss_trn: 2.326749e-01, elps: 4.022
ep: 20, loss_trn: 2.244175e-01, elps: 4.018
ep: 30, loss_trn: 2.199011e-01, elps: 4.065


KeyboardInterrupt: 

In [ ]:
Shiba_PINN(tf.zeros([100,3]))
Shiba_PINN.summary()

Model: "shiba_pinn_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 3)]               0         
_________________________________________________________________
layer1 (Dense)               multiple                  80        
_________________________________________________________________
layer2 (Dense)               multiple                  420       
_________________________________________________________________
layer3 (Dense)               multiple                  420       
_________________________________________________________________
layer4 (Dense)               multiple                  420       
_________________________________________________________________
layer5 (Dense)               multiple                  420       
_________________________________________________________________
layer6 (Dense)               multiple                